In [1]:
import numpy as np
import pandas as pd
import re
#visual
import matplotlib.pyplot as plt
import cv2 as cv
import scipy as sy
from PIL import Image

#file
import shutil
import os
import pathlib
import glob

#models
import tensorflow_hub as hub
import tensorflow as tf
import keras
from keras.applications.vgg16 import VGG16

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

from skimage.feature import greycomatrix,greycoprops
from skimage import io


/home/tama/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


### masking

In [2]:
SIZE = 256

train_image_mask= []
train_label_mask = []

for directory_path in glob.glob("../data/img_cyt_train/*"):
  label = directory_path.split("\\")[-1]
  #print()
  for img_path in glob.glob(os.path.join(directory_path,"*.bmp")):
    #print(img_path)
    img = cv.imread(img_path, cv.IMREAD_COLOR)
    img = cv.resize(img,(SIZE,SIZE))
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    train_image_mask.append(img)
    train_label_mask.append(label)

train_images_mask = np.array(train_image_mask)
train_labels_mask = np.array(train_label_mask)

In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels_mask)
train_label_endc_mask = le.transform(train_labels_mask)



x_train_mask, y_train_mask = train_images_mask, train_label_endc_mask

x_train_mask = x_train_mask/225

np.savetxt('mask/vgg/dense_label_train_mask.txt', y_train_mask)

In [4]:
keras.backend.clear_session()
vgg16_model_mask = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE,SIZE,3))
for layers in vgg16_model_mask.layers:
            layers.trainable=False
vgg16_model_mask.summary()

2022-11-02 01:44:44.430247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [9]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[:500], batch_size=10)
print(feature_extraction_mobile.shape)

2022-11-02 01:05:27.280932: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 393216000 exceeds 10% of free system memory.
2022-11-02 01:05:34.465582: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 167772160 exceeds 10% of free system memory.
2022-11-02 01:05:34.642578: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 167772160 exceeds 10% of free system memory.
2022-11-02 01:05:35.750051: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 41943040 exceeds 10% of free system memory.
2022-11-02 01:05:35.921930: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 83886080 exceeds 10% of free system memory.


10/50 [=====>........................] - ETA: 2:21

In [ ]:
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

(500, 62720)

In [ ]:
np.savetxt("mask/vgg/pre_train1.txt",feature_extraction_mobile)

In [ ]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[500:1000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 12s 247ms/step


(500, 62720)

In [ ]:
np.savetxt("mask/vgg/pre_train2.txt",feature_extraction_mobile)

In [ ]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[1000:1500], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 12s 247ms/step


(500, 62720)

In [ ]:
np.savetxt("mask/vgg/pre_train3.txt",feature_extraction_mobile)

In [ ]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[1500:2000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 12s 244ms/step


(500, 62720)

In [ ]:
np.savetxt("mask/vgg/pre_train4.txt",feature_extraction_mobile)

In [ ]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[2000:2500], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 12s 247ms/step


(500, 62720)

In [ ]:
np.savetxt("mask/vgg/pre_train5.txt",feature_extraction_mobile)

In [ ]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[2500:3000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 11s 222ms/step


(500, 62720)

In [ ]:
np.savetxt("mask/vgg/pre_train6.txt",feature_extraction_mobile)

In [5]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[3000:], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

2022-11-02 01:44:46.647411: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 823394304 exceeds 10% of free system memory.
2022-11-02 01:44:54.092186: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 167772160 exceeds 10% of free system memory.
2022-11-02 01:44:54.339198: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 167772160 exceeds 10% of free system memory.
2022-11-02 01:44:54.994327: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 41943040 exceeds 10% of free system memory.
2022-11-02 01:44:55.122249: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 83886080 exceeds 10% of free system memory.


105/105 [==============================] - 380s 4s/step


(1047, 32768)

In [6]:
np.savetxt("mask/vgg/pre_train7.txt",feature_extraction_mobile)

In [ ]:
#dt_DenseNet1 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train1.txt')
#dt_DenseNet2 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train2.txt')
#dt_DenseNet3 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train3.txt')
#dt_DenseNet4 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train4.txt')
#dt_DenseNet5 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train5.txt')
#dt_DenseNet6 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train6.txt')
#dt_DenseNet7 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train7.txt')

#DenseNet_full = [dt_DenseNet1,dt_DenseNet2,dt_DenseNet3,dt_DenseNet4,dt_DenseNet5,dt_DenseNet6,dt_DenseNet7]

#for i in DenseNet_full:print(i.shape)

In [ ]:
#train_concat = np.concatenate((dt_DenseNet1, dt_DenseNet2, dt_DenseNet3, dt_DenseNet4, dt_DenseNet5, dt_DenseNet6), axis=0)
#np.savetxt("mask/DenseNet121/densenet_full_data.txt",train_concat)

### no mask

In [2]:
SIZE = 256

train_image_mask= []
train_label_mask = []

for directory_path in glob.glob("../data/train/*"):
  label = directory_path.split("\\")[-1]
  #print()
  for img_path in glob.glob(os.path.join(directory_path,"*.bmp")):
    #print(img_path)
    img = cv.imread(img_path, cv.IMREAD_COLOR)
    img = cv.resize(img,(SIZE,SIZE))
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    train_image_mask.append(img)
    train_label_mask.append(label)

train_images_mask = np.array(train_image_mask)
train_labels_mask = np.array(train_label_mask)

In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels_mask)
train_label_endc_mask = le.transform(train_labels_mask)



x_train_mask, y_train_mask = train_images_mask, train_label_endc_mask

x_train_mask = x_train_mask/225

np.savetxt('nomask/vgg/dense_label_train_mask.txt', y_train_mask)

In [4]:
keras.backend.clear_session()
vgg16_model_mask = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE,SIZE,3))
for layers in vgg16_model_mask.layers:
            layers.trainable=False
vgg16_model_mask.summary()

2023-01-17 12:33:10.650123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [5]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[:500], batch_size=10)
print(feature_extraction_mobile.shape)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

np.savetxt("nomask/vgg/pre_train1.txt",feature_extraction_mobile)

2023-01-17 12:34:35.192319: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 393216000 exceeds 10% of free system memory.
2023-01-17 12:34:36.315653: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 167772160 exceeds 10% of free system memory.
2023-01-17 12:34:36.486285: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 167772160 exceeds 10% of free system memory.
2023-01-17 12:34:37.076616: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 41943040 exceeds 10% of free system memory.
2023-01-17 12:34:37.347542: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 83886080 exceeds 10% of free system memory.


50/50 [==============================] - 204s 4s/step
(500, 8, 8, 512)


In [6]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[500:1000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

np.savetxt("nomask/vgg/pre_train2.txt",feature_extraction_mobile)

50/50 [==============================] - 202s 4s/step


In [7]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[1000:1500], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

np.savetxt("nomask/vgg/pre_train3.txt",feature_extraction_mobile)

50/50 [==============================] - 246s 5s/step


In [8]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[1500:2000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

np.savetxt("nomask/vgg/pre_train4.txt",feature_extraction_mobile)

50/50 [==============================] - 207s 4s/step


In [9]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[2000:2500], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

np.savetxt("nomask/vgg/pre_train5.txt",feature_extraction_mobile)

50/50 [==============================] - 204s 4s/step


In [10]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[2500:3000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

np.savetxt("nomask/vgg/pre_train6.txt",feature_extraction_mobile)

50/50 [==============================] - 209s 4s/step


In [11]:
feature_extraction_mobile = vgg16_model_mask.predict(x_train_mask[3000:], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

np.savetxt("nomask/vgg/pre_train7.txt",feature_extraction_mobile)

4/4 [==============================] - 14s 3s/step


In [2]:
dt_DenseNet1 = np.loadtxt('nomask/vgg/pre_train1.txt')
dt_DenseNet2 = np.loadtxt('nomask/vgg/pre_train2.txt')
dt_DenseNet3 = np.loadtxt('nomask/vgg/pre_train3.txt')
dt_DenseNet4 = np.loadtxt('nomask/vgg/pre_train4.txt')
dt_DenseNet5 = np.loadtxt('nomask/vgg/pre_train5.txt')
dt_DenseNet6 = np.loadtxt('nomask/vgg/pre_train6.txt')
dt_DenseNet7 = np.loadtxt('nomask/vgg/pre_train7.txt')

DenseNet_full = [dt_DenseNet1,dt_DenseNet2,dt_DenseNet3,dt_DenseNet4,dt_DenseNet5,dt_DenseNet6,dt_DenseNet7]

for i in DenseNet_full:print(i.shape)

(500, 32768)
(500, 32768)
(500, 32768)
(500, 32768)
(500, 32768)
(500, 32768)
(34, 32768)


In [3]:
train_concat = np.concatenate((dt_DenseNet1, dt_DenseNet2, dt_DenseNet3, dt_DenseNet4, dt_DenseNet5, dt_DenseNet6), axis=0)
np.savetxt("nomask/vgg/vgg_full_data.txt",train_concat)